In [1]:
!pip install noisereduce
!pip install praat-parselmouth
!pip install --upgrade librosa
!pip install librosa scipy
!pip install pysptk
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.9/461.9 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pysptk: filename=pysptk-1.0.1-cp310-cp310-linux_x86_64.whl size=1241912 sha256=5e0102debad2fe941dac697bfddf4e39de292b7e6c5de7afd97db01e22b769c0
  Stored in directory: /root/.cache/pip/wheels/ba/c2/7f/73c54e1795eeca8f995f95474c8290b78ee0a6306412dc934f
Successfully built pysptk


In [2]:
import numpy as np
import os
import pandas as pd
import re
import librosa
import parselmouth
from parselmouth.praat import call
import scipy.stats as stats
import scipy
import pysptk
from scipy.io import wavfile
from tqdm import tqdm
from google.colab import drive

# Scikit-learn imports
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# Proceed with splitting the data and training the model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# Google Colab specific
drive.mount('/content/drive')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive


In [3]:
import re
import nltk
from transformers import BertTokenizer, BertModel
import torch
from torch import nn, optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Download NLTK tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_csv("/content/drive/Shared drives/CS298/CS 297/IEMOCAP/IEMOCAP_MAJOR_LABELLED.csv")

In [ ]:
df.head()

,Session,Conversation,Audio_Path,Speaker_id,Transcript,Time_Frame,Audio_Uttrance_Path,category,most_repeated_label
0,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M001,I wonder.,008.6300-009.6700,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Fear', 'Excited']",fear
1,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M002,It must be them then.,011.4300-012.7900,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Fear', 'Excited']",fear
2,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_F005,Ahh. It sent shivers up my spine.,025.9111-030.4663,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Frustration', 'Excited', 'Excited']",excited
3,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M006,What shall we do if they suddenly walk in on us?,029.8095-033.1781,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Frustration', 'Excited', 'Excited']",excited
4,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_F006,Behave exquisitely.,032.8800-034.9200,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Excited', 'Happiness', 'Excited']",excited


In [ ]:
print(type(df['most_repeated_label'].iloc[0]))

<class 'str'>


In [ ]:
import ast

# Apply ast.literal_eval to convert string to list
df['category'] = df['category'].apply(lambda x: ast.literal_eval(x))
print(type(df['category'].iloc[0]))


<class 'list'>


In [ ]:
df['category'].iloc[0]

['Frustration', 'Fear', 'Excited']

In [ ]:
import pandas as pd
from collections import Counter

# Step 1: Copy df to new_df
new_df = df.copy()

# Step 2: Function to find the most repeated label (case insensitive)
def find_most_repeated_label(label_list):
    # Convert all items to lowercase
    lower_case_labels = [label.lower() for label in label_list]

    # Count the occurrences of each label
    label_counts = Counter(lower_case_labels)

    # Find the label(s) that repeat more than once
    most_common_label, count = label_counts.most_common(1)[0]  # Get the most common label

    # If it repeats more than once, return it, otherwise return None
    if count > 1:
        return most_common_label
    else:
        return None

# Step 3: Apply the function to each row in the 'category' column and create a new column 'most_repeated_label'
new_df['most_repeated_label'] = new_df['category'].apply(lambda x: find_most_repeated_label(x))


In [ ]:
new_df.head()

,Session,Conversation,Audio_Path,Speaker_id,Transcript,Time_Frame,Major_emotion,Audio_Uttrance_Path,Major_Emotion,Major,...,Dominance,dimension,category,act,val,dom,New_Category,Final_class,Unnamed: 20,most_repeated_label
0,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M000,Good God.,006.2551-008.1400,Frustration,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Frustration']",Fear,...,"[4, 4]","[{'arousal': 3.0, 'valence': 3.0, 'dominance':...","[Frustration, Fear, Excited]","2,4","3,3","4,4",Nervous,NaN,NaN,None
1,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_F000,Do you think it's them?,007.7500-009.1900,Frustration,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Frustration']",Fear,...,"[3, 4]","[{'arousal': 3.5, 'valence': 2.0, 'dominance':...","[Frustration, Fear, Excited]","2,5","2,2","3,4",Nervous,NaN,NaN,None
2,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M001,I wonder.,008.6300-009.6700,Fear,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Fear']",Fear,...,"[3, 4]","[{'arousal': 3.0, 'valence': 3.0, 'dominance':...","[Fear, Fear, Excited]","2,4","3,3","3,4",Nervous,NaN,NaN,fear
3,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_F001,Nobody knows we're here except Freda and she w...,009.2300-011.8600,Frustration,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Frustration']",Fear,...,"[3, 4]","[{'arousal': 4.0, 'valence': 2.5, 'dominance':...","[Frustration, Fear, Excited]","3,5","3,2","3,4",Nervous,NaN,NaN,None
4,Session4,Ses04F_script03_1,/content/drive/My Drive/CS 297/IEMOCAP/Session...,Ses04F_script03_1_M002,It must be them then.,011.4300-012.7900,Fear,/content/drive/My Drive/CS 297/IEMOCAP/Session...,"['Fear', 'Excited', 'Fear']",Fear,...,"[3, 4]","[{'arousal': 3.0, 'valence': 2.5, 'dominance':...","[Fear, Fear, Excited]","2,4","2,3","3,4",Nervous,NaN,NaN,fear


In [ ]:
new_df = new_df.drop(columns=['act', 'val', 'dom', 'New_Category', 'Final_class', 'Unnamed: 20'])


In [ ]:
new_df = new_df.drop(columns=['Major_Emotion', 'Major_emotion','Major', 'Arousal', 'Valence', 'Dominance', 'dimension'])


In [ ]:
new_df = new_df[new_df['most_repeated_label'].notna()]

In [ ]:
len(new_df)

7766

In [ ]:
label_counts = new_df['most_repeated_label'].value_counts()

print(label_counts)

most_repeated_label
frustration    2026
neutral        1710
excited        1174
sadness        1075
anger          1042
happiness       586
surprise        102
fear             34
other            14
disgust           3
Name: count, dtype: int64


In [ ]:
import pandas as pd
new_df = df.copy()
# Modify the labels according to the merging criteria
new_df['most_repeated_label'] = new_df['most_repeated_label'].replace({
    #'frustration': 'anger',   # Merge 'frustration' into 'anger'
    'excited': 'happy',       # Merge 'excited' into 'happy'
    'happiness': 'happy'      # Merge 'happiness' into 'happy'
})

# Drop rows with the labels: 'surprise', 'fear', 'other', and 'disgust'
new_df = new_df[~new_df['most_repeated_label'].isin(['surprise', 'fear', 'other', 'disgust','frustration'])]

# Print updated value counts to verify
print(new_df['most_repeated_label'].value_counts())


most_repeated_label
happy      1760
neutral    1710
sadness    1075
anger      1042
Name: count, dtype: int64


In [ ]:
#new_df.to_csv('IEMOCAP_MAJOR_LABELLED.csv', index=False)

In [ ]:
# Preprocessing function
def preprocess(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = nltk.word_tokenize(text)  # Tokenize text
    return " ".join(tokens)

In [ ]:
new_df['cleaned_text'] = new_df['Transcript'].apply(preprocess)

In [ ]:
# Encode the target labels
label_encoder = LabelEncoder()
new_df['encoded_label'] = label_encoder.fit_transform(new_df['most_repeated_label'])

In [ ]:
from tqdm import tqdm
#BERT tokenizer to convert text to BERT-compatible tokens
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class EmotionDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Define the model
class BERTEmotionClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BERTEmotionClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )['pooler_output']
        output = self.drop(pooled_output)
        return self.out(output)



# Training function
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    correct_predictions = 0
    loop = tqdm(data_loader, leave=False)

    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = loss_fn(outputs, labels)
        _, preds = torch.max(outputs, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Update tqdm description with current loss
        loop.set_description(f"Loss: {loss.item():.4f}")

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)



# Evaluation
def eval_model(model, data_loader, device):
    model = model.eval()
    correct_predictions = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            correct_predictions += torch.sum(preds == labels)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
        # Calculate accuracy
        accuracy = correct_predictions.double() / len(data_loader.dataset)

        # Calculate F1 score (macro or weighted depending on your needs)
        f1 = f1_score(true_labels, predictions, average='weighted')

    return accuracy, f1, predictions, true_labels


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Split the data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(new_df['cleaned_text'], new_df['encoded_label'], test_size=0.2, random_state=42)

train_dataset = EmotionDataset(train_texts.tolist(), train_labels.tolist())
test_dataset = EmotionDataset(test_texts.tolist(), test_labels.tolist())

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4)

In [ ]:
# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTEmotionClassifier(n_classes=len(label_encoder.classes_))
model = model.to(device)

# Loss function and optimizer
optimizer = optim.Adam(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
for epoch in range(5):  # For demo, you can increase this for better performance
    print(f'Epoch {epoch + 1}/{2}')
    train_acc, train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device)
    print(f'Train accuracy: {train_acc}, Loss: {train_loss}')

Epoch 1/2


Train accuracy: 0.5390467666144552, Loss: 1.0794441892044062
Epoch 2/2


Train accuracy: 0.7531886328037593, Loss: 0.6370287163346366
Epoch 3/2


Train accuracy: 0.8435891698366524, Loss: 0.42624772012680234
Epoch 4/2


Train accuracy: 0.876706198254643, Loss: 0.31989011532283007
Epoch 5/2


Train accuracy: 0.895949876929962, Loss: 0.2659812551658194


In [ ]:
# Evaluate on test set
test_acc, preds, true_labels = eval_model(model, test_loader, device)
print(f'Test Accuracy: {test_acc}')

# Display classification report
from sklearn.metrics import classification_report
print(classification_report(true_labels, preds, target_names=label_encoder.classes_))

Test Accuracy: 0.6762075134168157
              precision    recall  f1-score   support

       anger       0.73      0.68      0.71       193
       happy       0.67      0.80      0.73       360
     neutral       0.60      0.58      0.59       333
     sadness       0.76      0.61      0.68       232

    accuracy                           0.68      1118
   macro avg       0.69      0.67      0.68      1118
weighted avg       0.68      0.68      0.67      1118



In [ ]:
# Evaluate on test set
test_acc, preds, true_labels = eval_model(model, test_loader, device)
print(f'Test Accuracy: {test_acc}')

# Display classification report
from sklearn.metrics import classification_report
print(classification_report(true_labels, preds, target_names=label_encoder.classes_))

Test Accuracy: 0.6762075134168157
              precision    recall  f1-score   support

       anger       0.73      0.68      0.71       193
       happy       0.67      0.80      0.73       360
     neutral       0.60      0.58      0.59       333
     sadness       0.76      0.61      0.68       232

    accuracy                           0.68      1118
   macro avg       0.69      0.67      0.68      1118
weighted avg       0.68      0.68      0.67      1118



In [ ]:
from sklearn.metrics import confusion_matrix

# Number of folds
n_splits = 5

# Initialize KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Create a list to store results for each fold
fold_accuracies = []
fold_f1 = []
# Convert text and labels to lists for cross-validation purposes
texts = new_df['cleaned_text'].tolist()
labels = new_df['encoded_label'].tolist()

# Check the size of the data
print(f"Number of samples in texts: {len(texts)}")
print(f"Number of samples in labels: {len(labels)}")
num_epoch = 10
# Begin 5-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(texts)):
    print(f"Fold {fold+1}/{n_splits}")

    # Debug: Check the sizes of the indices
    print(f"Train indices size: {len(train_index)}, Val indices size: {len(val_index)}")

    # Split the data into training and validation sets for this fold
    try:
        train_texts = [texts[i] for i in train_index]
        val_texts = [texts[i] for i in val_index]
        train_labels = [labels[i] for i in train_index]
        val_labels = [labels[i] for i in val_index]
    except IndexError as e:
        print(f"IndexError: {e}")
        print(f"Text size: {len(texts)}, Label size: {len(labels)}")
        break  # Break the loop if there's an issue with indexing

    # Create the datasets and dataloaders for this fold
    train_dataset = EmotionDataset(train_texts, train_labels)
    val_dataset = EmotionDataset(val_texts, val_labels)

    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=4)

    # Reinitialize the model for each fold
    model = BERTEmotionClassifier(n_classes=len(label_encoder.classes_))
    model = model.to(device)

    # Reinitialize the optimizer and loss function for each fold
    optimizer = optim.Adam(model.parameters(), lr=2e-5)
    loss_fn = nn.CrossEntropyLoss()

    # Train for a few epochs (adjust as necessary)
    for epoch in range(10):  # Adjust epochs as needed
        print(f'Epoch {epoch + 1}/3')

        # Training step
        train_acc, train_loss = train_epoch(model, train_loader, loss_fn, optimizer, device)

        # Validation step
        val_acc, val_f1, val_preds, val_true_labels = eval_model(model, val_loader, device)

        print(f'Train accuracy: {train_acc}, Train loss: {train_loss}')
        print(f'Validation accuracy: {val_acc}')

    # Generate and print confusion matrix for this fold
    cm = confusion_matrix(val_true_labels, val_preds)
    print(f'Confusion Matrix for Fold {fold + 1}:\n', cm)


    # Store the validation accuracy for this fold
    fold_accuracies.append(val_acc.item())
    fold_f1.append(val_f1)
    print('********************************************')
# After all folds are completed, calculate and display the average results
avg_accuracy = np.mean(fold_accuracies)
avg_f1 = np.mean(fold_f1)
print(f"\nAverage Validation Accuracy across {n_splits} folds: {avg_accuracy}")
print(f"\nAverage Validation F1 across {n_splits} folds: {avg_f1}")



Number of samples in texts: 5587
Number of samples in labels: 5587
Fold 1/5
Train indices size: 4469, Val indices size: 1118
Epoch 1/3


Train accuracy: 0.564779592750056, Train loss: 1.036753500336931
Validation accuracy: 0.6699463327370304
Epoch 2/3


Train accuracy: 0.7719847840680242, Train loss: 0.612238131635131
Validation accuracy: 0.7048300536672629
Epoch 3/3


Train accuracy: 0.8534347728798388, Train loss: 0.4016581814251307
Validation accuracy: 0.6976744186046512
Epoch 4/3


Train accuracy: 0.8878943835309913, Train loss: 0.30245989760007436
Validation accuracy: 0.6994633273703041
Epoch 5/3


...

Train accuracy: 0.9105145413870246, Train loss: 0.22304242111612127
Validation accuracy: 0.7045658012533572
Epoch 7/3


Train accuracy: 0.90917225950783, Train loss: 0.21911503788465317
Validation accuracy: 0.6866606982990152
Epoch 8/3


Train accuracy: 0.9201342281879195, Train loss: 0.19471899123450245
Validation accuracy: 0.702775290957923
Epoch 9/3


Train accuracy: 0.9190156599552572, Train loss: 0.1896347856435534
Validation accuracy: 0.7009847806624888
Epoch 10/3


Train accuracy: 0.9178970917225951, Train loss: 0.19112601329851353
Validation accuracy: 0.6866606982990152
Confusion Matrix for Fold 5:
 [[174  10  26  25]
 [ 13 252  38  36]
 [ 36  52 178  61]
 [  6  25  22 163]]
********************************************

Average Validation Accuracy across 5 folds: 0.7000182574395062

Average Validation F1 across 5 folds: 0.6991866897078287
